## XGBSurv Breslow Benchmark

In [1]:
from xgbsurv.datasets import (load_metabric, load_flchain, load_rgbsg, load_support, load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
# import models
from xgbsurv.models.breslow_final import breslow_likelihood,get_cumulative_hazard_function_breslow
from xgbsurv.models.efron_final import efron_likelihood
from xgbsurv.models.cind_final import cind_loss
from xgbsurv.models.deephit_pycox_final import deephit_loss1_pycox
from xgbsurv.models.eh_aft_final import aft_likelihood
from xgbsurv.models.eh_ah_final import ah_likelihood
from pycox.evaluation import EvalSurv
from sklearn.utils.fixes import loguniform
from xgbsurv.models.utils import sort_X_y_pandas, transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
import os
import sys
current_path = os.getcwd() 
one_level_up = os.path.abspath(os.path.join(current_path,  ".."))
two_levels_up = os.path.abspath(os.path.join(current_path,  "..",".."))
sys.path.append(one_level_up+'/gbdt_pipeline')
from gbdt_pipeline import get_gbdt_pipeline, train_gbdt, train_gbdt_complete

current path /Users/JUSC/Documents/xgbsurv/experiments/ah


## Set Parameters

In [2]:
# set parameters
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 50
early_stopping_rounds=10
base_score = 0.0
validation_size = 0.2
method = '_gbdt'


# set seed for scipy
np.random.seed(rand_state)


## Custom Splitting

In [3]:
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        try:
            if y.shape[1]>1:
                y = y[:,0]
        except:
            pass
        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)


## Training

In [4]:
data_set_fns = [load_metabric,  load_flchain, load_rgbsg, load_support] 
metrics_list = []

for idx, dataset in enumerate(data_set_fns):
    model = 'ah'
    # get name of current dataset
    data = dataset(path=two_levels_up+"/xgbsurv/datasets/data/", as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_gbdt_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter)
        print(metric)
        metrics_list.append(metric)



METABRIC_adapted.csv
0
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
integration values 208367
integration values 34667
Concordance Index 0.5960279353993889
Integrated Brier Score: 0.19515077806958922
{'model': 'ah', 'dataset': 'METABRIC', 'cindex_train': [0.6220177045100778], 'cindex_test': [0.5960279353993889], 'ibs_train': [0.1772077438139772], 'ibs_test': [0.19515077806958922]}
1
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
integration values 167931
integration values 29618
Concordance Index 0.5572774292513127
Integrated Brier Score: 0.18578269722467403
{'model': 'ah', 'dataset': 'METABRIC', 'cindex_train': [0.5739909808766047], 'cindex_test': [0.5572774292513127], 'ibs_train': [0.1794880075358399], 'ibs_test': [0.18578269722467403]}
2
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
integration va

In [5]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_results.csv',index=False)

In [6]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,ah,METABRIC,0.622018,0.596028,0.177208,0.195151
1,ah,METABRIC,0.573991,0.557277,0.179488,0.185783
2,ah,METABRIC,0.613371,0.547251,0.178285,0.198259
3,ah,METABRIC,0.586309,0.577820,0.182801,0.176047
4,ah,METABRIC,0.593176,0.565580,0.180656,0.182825
5,ah,FLCHAIN,0.544277,0.522738,0.131845,0.136329
6,ah,FLCHAIN,0.894180,0.720000,0.044918,0.115782
7,ah,FLCHAIN,0.929042,0.701564,0.045297,0.118711
8,ah,FLCHAIN,0.618526,0.546870,0.129586,0.130296
9,ah,FLCHAIN,0.886532,0.708270,0.044865,0.119586


## TCGA Train, Test, Evaluation

In [7]:
cancer_types = [
    'BLCA',
    'BRCA',
    'HNSC',
    'KIRC',
    'LGG',
    'LIHC',
    'LUAD',
    'LUSC',
    'OV',
    'STAD'
    ]

metrics_list = []

for i, cancer_type in enumerate(cancer_types):
    model = 'ah'
    # get name of current dataset
    data = load_tcga(path=two_levels_up+"/xgbsurv/datasets/data/",cancer_type=cancer_type, as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_gbdt_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter, tcga=True)
        print(metric)
        metrics_list.append(metric)

BLCA_adapted.csv
0
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
integration values 15229
integration values 1956
Concordance Index 0.5917127071823205
Integrated Brier Score: 0.25956713393261027
{'model': 'ah', 'dataset': 'BLCA', 'cindex_train': [0.6539695416497658], 'cindex_test': [0.5917127071823205], 'ibs_train': [0.17906770411442732], 'ibs_test': [0.25956713393261027]}
1
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
integration values 12896
integration values 1591
Concordance Index 0.4516657564172583
Integrated Brier Score: 0.23185081334822202
{'model': 'ah', 'dataset': 'BLCA', 'cindex_train': [0.6256353181227973], 'cindex_test': [0.4516657564172583], 'ibs_train': [0.1945048081241711], 'ibs_test': [0.23185081334822202]}
2
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
integration values 10375
inte

In [8]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_tcga_results.csv',index=False)

In [9]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,ah,BLCA,0.653970,0.591713,0.179068,0.259567
1,ah,BLCA,0.625635,0.451666,0.194505,0.231851
2,ah,BLCA,0.585469,0.405018,0.200013,0.218289
3,ah,BLCA,0.303976,0.203895,0.213159,0.227086
4,ah,BLCA,0.577080,0.437621,0.204463,0.218299
5,ah,BRCA,0.629999,0.521701,0.180034,0.208756
6,ah,BRCA,0.245748,0.074195,0.197621,0.201300
7,ah,BRCA,0.615069,0.539179,0.180692,0.196929
8,ah,BRCA,0.717726,0.444754,0.195062,0.183910
9,ah,BRCA,0.440383,0.411489,0.195042,0.219803
